In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from IPython.display import Image

tf.random.set_seed(678)

## 다층퍼셉트론 구조
텐서플로우로 아래의 다층퍼셉트론을 구현해보도록 한다.

In [2]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/dropout.png", width=500, height=250)

# MNIST 데이터 불러오기

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [6]:
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) + " * " + str(x_train.shape[2])+ " image")

train data has 60000 samples
every train data is 28 * 28 image


데이터는 흑백 이미지로 0에 가까울수록 흰색, 255에 가까울수록 검은색 픽셀이다.

In [7]:
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


아래 코드를 실행하여 각 데이터에 해당하는 레이블이 0부터 9까지의 숫자임을 확인할 수 있다.

In [8]:
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


In [9]:
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

test data has 10000 samples
every test data is 28 * 28 image


## 데이터 정규화
데이터 정규화는 보통 학습 시간을 단축하고, 더 나은 성능을 구하도록 도와준다.  
MNIST 데이터의 모든 값은 0부터 255 범위 안에 있으므로 255로 나눔으로써 모든 값을 0과 1사이의 값으로 정규화한다.  

In [11]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

# 텐서플로우 다층퍼셉트론 구현하기

다층 퍼셉트론의 입력 레이어에 데이터를 넣기 위해 2d tensor(28,28)인 데이터를 1d tensor(28*28, 1)의 형태러ㅗ 바꾼다.  
이는 행렬 형태의 데이터를 배열 형태의 데이터로 변경한다는 의미와 같다.  

In [12]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/reshape_mnist.png", width=300, height=150)

In [13]:
model = Sequential([
    Flatten(input_shape=(28,28)),   # 데이터 차원 변경
    Dense(256, activation='relu'),  # 첫번째 히든 레이더(h1)
    Dense(128, activation='relu'),  # 두번째 히든 레이어(h2)
    Dropout(0.1),  # 두번째 히든 레이어(h2)에 드롭아웃(10%) 적용
    Dense(10),     # 세번째 히든 레이어 (logit)
    Activation('softmax')  # softmax layer
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
__________________________________________________

첫번째 레이어에 784개의 입력을 받는 256개의 노드가 존재하고, 노드마다 편향값이 하나씩 존재하므로  
784 * 256 + 256 = 200960의 파라미터가 존재한다.  
flatten과 softmax는 노드가 없으므로 파라미터가 존재하지 않는 것을 확인할 수 있다.

In [15]:
"""
sparse_categorical_crossentropy:
레이블을 원 핫 인코딩으로 자동으로 변경하여 크로스 엔트로피 측정합니다.
"""
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 조기종료(Early Stopping)
매 주기(Epoch)마다 검증 데이터로 검증 정확도를 측정한다.  
검증 정확도가 5번 연속으로 개선되지 않을 시, 조기 종료를 수행한다.

In [16]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=False),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

In [17]:
model.fit(x_train, y_train, epochs=300, batch_size=1000, validation_split=0.1, callbacks=callbacks)

Epoch 1/300
54/54 [==============================] - 1s 19ms/step - loss: 1.1574 - accuracy: 0.6674 - val_loss: 0.2151 - val_accuracy: 0.9403
Epoch 2/300
54/54 [==============================] - 0s 8ms/step - loss: 0.2617 - accuracy: 0.9243 - val_loss: 0.1459 - val_accuracy: 0.9638
Epoch 3/300
54/54 [==============================] - 0s 8ms/step - loss: 0.1792 - accuracy: 0.9488 - val_loss: 0.1238 - val_accuracy: 0.9665
Epoch 4/300
54/54 [==============================] - 0s 8ms/step - loss: 0.1352 - accuracy: 0.9605 - val_loss: 0.1038 - val_accuracy: 0.9688
Epoch 5/300
54/54 [==============================] - 0s 8ms/step - loss: 0.1129 - accuracy: 0.9680 - val_loss: 0.0910 - val_accuracy: 0.9730
Epoch 6/300
54/54 [==============================] - 0s 8ms/step - loss: 0.0950 - accuracy: 0.9718 - val_loss: 0.0867 - val_accuracy: 0.9742
Epoch 7/300
54/54 [==============================] - 0s 8ms/step - loss: 0.0756 - accuracy: 0.9790 - val_loss: 0.0807 - val_accuracy: 0.9758
Epoch 8/300


## 테스트

In [18]:
results = model.evaluate(x_test, y_test, verbose =0)

In [19]:
print('test loss, test acc : ', results)

test loss, test acc :  [0.0741107165813446, 0.9797999858856201]
